In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7681,2021-02-23T17:00:00,ITA,19,Sicilia,38.115697,13.362357,818,135,953,27704,...,NaN,5.0,NaN,NaN,149854.0,0.0,1705322.0,610594.0,ITG,ITG1
7682,2021-02-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,824,159,983,14138,...,NaN,16.0,NaN,NaN,150029.0,527.0,2393012.0,225809.0,ITI,ITI1
7683,2021-02-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,473,79,552,7957,...,"Si fa presente che 10 dei ricoveri NON UTI, so...",5.0,NaN,NaN,43351.0,0.0,676859.0,105817.0,ITI,ITI2
7684,2021-02-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,6,0,6,130,...,NaN,0.0,NaN,NaN,7982.0,0.0,73151.0,2954.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7681,2021-02-23T17:00:00,19,Sicilia,818,135,953,27704,28657,-710,452,...,149854,2315916,1104845.0,5.0,149854.0,0.0,1705322.0,610594.0,ITG,ITG1
7682,2021-02-23T17:00:00,9,Toscana,824,159,983,14138,15121,334,824,...,150556,2618821,1386367.0,16.0,150029.0,527.0,2393012.0,225809.0,ITI,ITI1
7683,2021-02-23T17:00:00,10,Umbria,473,79,552,7957,8509,35,262,...,43351,782676,309492.0,5.0,43351.0,0.0,676859.0,105817.0,ITI,ITI2
7684,2021-02-23T17:00:00,2,Valle d'Aosta,6,0,6,130,136,-10,4,...,7982,76105,43569.0,0.0,7982.0,0.0,73151.0,2954.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-23', '2021-02-22')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-23T17:00:00,1,Piemonte,1919,145,2064,11041,13105,185,1023,...,240611,2468673,1217712.0,12.0,236183.0,4428.0,2032975.0,435698.0,ITC,ITC1
1,2021-02-23T17:00:00,2,Valle d'Aosta,6,0,6,130,136,-10,4,...,7982,76105,43569.0,0.0,7982.0,0.0,73151.0,2954.0,ITC,ITC2
2,2021-02-23T17:00:00,3,Lombardia,3917,408,4325,50390,54715,1346,2480,...,583251,6418047,3049391.0,29.0,577257.0,5994.0,6079928.0,338119.0,ITC,ITC4
3,2021-02-23T17:00:00,5,Veneto,806,105,911,21386,22297,13,1062,...,328078,4846574,1381843.0,15.0,324952.0,3126.0,3979200.0,867374.0,ITH,ITH3
4,2021-02-23T17:00:00,6,Friuli Venezia Giulia,353,57,410,8761,9171,-120,241,...,74349,1309050,467090.0,5.0,69948.0,4401.0,1224938.0,84112.0,ITH,ITH4
5,2021-02-23T17:00:00,7,Liguria,498,53,551,4316,4867,-131,381,...,76291,999439,427053.0,1.0,76291.0,0.0,906363.0,93076.0,ITC,ITC3
6,2021-02-23T17:00:00,8,Emilia-Romagna,2043,196,2239,34486,36725,209,1588,...,249429,3671610,1468388.0,17.0,249318.0,111.0,3310327.0,361283.0,ITH,ITH5
7,2021-02-23T17:00:00,9,Toscana,824,159,983,14138,15121,334,824,...,150556,2618821,1386367.0,16.0,150029.0,527.0,2393012.0,225809.0,ITI,ITI1
8,2021-02-23T17:00:00,10,Umbria,473,79,552,7957,8509,35,262,...,43351,782676,309492.0,5.0,43351.0,0.0,676859.0,105817.0,ITI,ITI2
9,2021-02-23T17:00:00,11,Marche,555,78,633,8491,9124,215,581,...,64437,801796,486227.0,6.0,64437.0,0.0,745611.0,56185.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-22T17:00:00,1,Piemonte,1905,141,2046,10874,12920,100,639,...,239588,2446789,1210042.0,13.0,235415.0,4173.0,2024542.0,422247.0,ITC,ITC1
1,2021-02-22T17:00:00,2,Valle d'Aosta,6,1,7,139,146,13,14,...,7978,75772,43431.0,0.0,7978.0,0.0,72972.0,2800.0,ITC,ITC2
2,2021-02-22T17:00:00,3,Lombardia,3826,391,4217,49152,53369,-1676,1491,...,580771,6382898,3039913.0,26.0,575328.0,5443.0,6057388.0,325510.0,ITC,ITC4
3,2021-02-22T17:00:00,5,Veneto,772,103,875,21409,22284,199,509,...,327016,4808969,1378883.0,6.0,323890.0,3126.0,3969322.0,839647.0,ITH,ITH3
4,2021-02-22T17:00:00,6,Friuli Venezia Giulia,350,56,406,8885,9291,17,145,...,74108,1299729,464867.0,4.0,69796.0,4312.0,1219261.0,80468.0,ITH,ITH4
5,2021-02-22T17:00:00,7,Liguria,515,56,571,4427,4998,108,136,...,75910,991704,423907.0,3.0,75910.0,0.0,901295.0,90409.0,ITC,ITC3
6,2021-02-22T17:00:00,8,Emilia-Romagna,2017,194,2211,34305,36516,488,1847,...,247856,3636600,1463743.0,17.0,247747.0,109.0,3292245.0,344355.0,ITH,ITH5
7,2021-02-22T17:00:00,9,Toscana,797,155,952,13835,14787,572,911,...,149732,2599096,1377334.0,11.0,149235.0,497.0,2381623.0,217473.0,ITI,ITI1
8,2021-02-22T17:00:00,10,Umbria,475,81,556,7918,8474,-74,97,...,43089,775431,308361.0,5.0,43089.0,0.0,672449.0,102982.0,ITI,ITI2
9,2021-02-22T17:00:00,11,Marche,543,78,621,8288,8909,-3,117,...,63856,794480,481285.0,11.0,63856.0,0.0,740904.0,53576.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-02-23T17:00:00,1,Piemonte,1919,145,2064,11041,13105,185,1023,...,2468673,1217712.0,12.0,236183.0,4428.0,2032975.0,435698.0,ITC,ITC1,4.341375e+06
1,2021-02-23T17:00:00,2,Valle d'Aosta,6,0,6,130,136,-10,4,...,76105,43569.0,0.0,7982.0,0.0,73151.0,2954.0,ITC,ITC2,1.255010e+05
2,2021-02-23T17:00:00,3,Lombardia,3917,408,4325,50390,54715,1346,2480,...,6418047,3049391.0,29.0,577257.0,5994.0,6079928.0,338119.0,ITC,ITC4,1.010397e+07
3,2021-02-23T17:00:00,5,Veneto,806,105,911,21386,22297,13,1062,...,4846574,1381843.0,15.0,324952.0,3126.0,3979200.0,867374.0,ITH,ITH3,4.907704e+06
4,2021-02-23T17:00:00,6,Friuli Venezia Giulia,353,57,410,8761,9171,-120,241,...,1309050,467090.0,5.0,69948.0,4401.0,1224938.0,84112.0,ITH,ITH4,1.211357e+06
5,2021-02-23T17:00:00,7,Liguria,498,53,551,4316,4867,-131,381,...,999439,427053.0,1.0,76291.0,0.0,906363.0,93076.0,ITC,ITC3,1.543127e+06
6,2021-02-23T17:00:00,8,Emilia-Romagna,2043,196,2239,34486,36725,209,1588,...,3671610,1468388.0,17.0,249318.0,111.0,3310327.0,361283.0,ITH,ITH5,4.467118e+06
7,2021-02-23T17:00:00,9,Toscana,824,159,983,14138,15121,334,824,...,2618821,1386367.0,16.0,150029.0,527.0,2393012.0,225809.0,ITI,ITI1,3.722729e+06
8,2021-02-23T17:00:00,10,Umbria,473,79,552,7957,8509,35,262,...,782676,309492.0,5.0,43351.0,0.0,676859.0,105817.0,ITI,ITI2,8.802850e+05
9,2021-02-23T17:00:00,11,Marche,555,78,633,8491,9124,215,581,...,801796,486227.0,6.0,64437.0,0.0,745611.0,56185.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-02-23T17:00:00,1,Piemonte,1919,145,2064,11041,13105,185,1023,...,435698.0,ITC,ITC1,4.341375e+06,Piemonte,1905,5824,141,628,99
1,2021-02-23T17:00:00,2,Valle d'Aosta,6,0,6,130,136,-10,4,...,2954.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,6,227,1,20,12
2,2021-02-23T17:00:00,3,Lombardia,3917,408,4325,50390,54715,1346,2480,...,338119.0,ITC,ITC4,1.010397e+07,Lombardia,3826,10355,391,1248,282
3,2021-02-23T17:00:00,5,Veneto,806,105,911,21386,22297,13,1062,...,867374.0,ITH,ITH3,4.907704e+06,Veneto,772,6000,103,1000,0
4,2021-02-23T17:00:00,6,Friuli Venezia Giulia,353,57,410,8761,9171,-120,241,...,84112.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,350,1277,56,175,0
5,2021-02-23T17:00:00,7,Liguria,498,53,551,4316,4867,-131,381,...,93076.0,ITC,ITC3,1.543127e+06,Liguria,515,1775,56,216,18
6,2021-02-23T17:00:00,8,Emilia-Romagna,2043,196,2239,34486,36725,209,1588,...,361283.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2017,6126,194,757,0
7,2021-02-23T17:00:00,9,Toscana,824,159,983,14138,15121,334,824,...,225809.0,ITI,ITI1,3.722729e+06,Toscana,797,5033,155,569,14
8,2021-02-23T17:00:00,10,Umbria,473,79,552,7957,8509,35,262,...,105817.0,ITI,ITI2,8.802850e+05,Umbria,475,851,81,141,2
9,2021-02-23T17:00:00,11,Marche,555,78,633,8491,9124,215,581,...,56185.0,ITI,ITI3,1.518400e+06,Marche,543,1176,78,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-23T17:00:00,3,Lombardia,3917,408,4325,50390,54715,1346,2480,...,32.69,1238,989,35149,9478.0,43,1091,7.06,0.02454,5.77
1,2021-02-23T17:00:00,8,Emilia-Romagna,2043,196,2239,34486,36725,209,1588,...,25.89,181,-259,35010,4645.0,25,1339,4.54,0.03555,5.58
2,2021-02-23T17:00:00,15,Campania,1327,132,1459,69074,70533,-719,1436,...,21.29,-734,234,15061,4031.0,38,2117,9.53,0.02482,4.42
3,2021-02-23T17:00:00,5,Veneto,806,105,911,21386,22297,13,1062,...,10.50,-23,553,37605,2960.0,21,1028,2.82,0.02164,6.68
4,2021-02-23T17:00:00,1,Piemonte,1919,145,2064,11041,13105,185,1023,...,23.09,167,384,21884,7670.0,22,816,4.67,0.02356,5.54
5,2021-02-23T17:00:00,12,Lazio,1850,229,2079,32848,34927,-283,889,...,24.28,-268,35,31628,20158.0,33,1139,2.81,0.01516,3.88
6,2021-02-23T17:00:00,9,Toscana,824,159,983,14138,15121,334,824,...,27.94,303,-87,19725,9033.0,20,470,4.18,0.02213,4.04
7,2021-02-23T17:00:00,16,Puglia,1265,164,1429,31013,32442,-253,823,...,28.82,-240,480,12067,5701.0,32,1044,6.82,0.02053,3.53
8,2021-02-23T17:00:00,11,Marche,555,78,633,8491,9124,215,581,...,33.48,203,464,7316,4942.0,9,357,7.94,0.03826,4.24
9,2021-02-23T17:00:00,19,Sicilia,818,135,953,27704,28657,-710,452,...,16.21,-678,40,25179,5000.0,21,1141,1.80,0.00910,3.02


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,79,62,141,56.03
1,Abruzzo,78,123,201,38.81
2,Molise,15,24,39,38.46
3,P.A. Trento,32,58,90,35.56
4,Marche,78,155,233,33.48
5,P.A. Bolzano,33,67,100,33.00
6,Lombardia,408,840,1248,32.69
7,Friuli Venezia Giulia,57,118,175,32.57
8,Puglia,164,405,569,28.82
9,Toscana,159,410,569,27.94


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))